In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# for more information about mounting a drive go to: 
# https://towardsdatascience.com/importing-data-to-google-colab-the-clean-way-5ceef9e9e3c8
#
# Quick how to:
#  - upload the magnetic field profile file (SlowerField.txt) and the atom
#    number/power measurement file (optical_tweezer_data.txt) into your google drive
#  - mount your drive with the code below.  It will ask you to follow a link to 
#    copy verification information
#  - import data following the example below
#
#       with open('/content/drive/My Drive/Colab Notebooks/105_Lab2/optical_tweezer_data.txt','r') as f:
#         tweezer_data=f.read()
#
#  - the data will be stored in tweezer_data as a string for .txt files
#  - use normal python trickery to extract the relevant data from the string
#  - if you get stuck or something isn't working, reach out to your Lab AI 
#  (Connor - connorh@princeton.edu, Fang - fangx@princeton.edu)

from google.colab import drive
drive.mount('/content/drive/')

from scipy.integrate import odeint

from scipy.optimize import curve_fit

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


For several of the exercises below, you are asked to derive formulae.  Please explain your derivations in markdown cells.  You can include mathematical expressions using LaTeX.  For further information on how to write equations in a markdown cell see <a href="https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html">this</a>. There are many online tutorials on using LaTeX notation.  

# Lab 2: Laser cooling and trapping of atoms

Most of this course is devoted to introducing you to classical mechanics in its Newtonian and Lagrangian formulations. Classical mechanics describes the motion of macroscopic objects. It is a subject that was primarily developed during the 17th and 18th centuries. Microscopic objects like electrons and atoms obey a different set of laws captured in the “quantum mechanics” developed in the early 20th century. In quantum mechanics, microscopic objects can exhibit wave-like phenomena like “superposition” and “interference”. You’ll understand the meanings of these words when we discuss classical waves in the last two weeks of this course and can get a first exposure to the quantum world by taking PHY 208. 

Although atoms are more complex than fundamental particles like electrons, they are in many ways easier to probe and manipulate. Lasers can be used to apply forces on atoms to cool and trap them. Cooling particles means slowing down their random thermal motion. When atoms are cold enough (typically at microkelvin temperatures), their wave-like character becomes important and an atomic gas can exhibit quantum mechanical phenomena. Cold atoms can also be trapped, i.e. held in place using electromagnetic fields. These trapped atoms can be used for many exciting applications including building ridiculously fast computers (quantum computers) or super accurate atomic clocks important for global navigation satellite systems. 

While you won’t be able to analyze the behavior of such systems at ultracold temperatures until you take a course in quantum mechanics, you can already understand much of the higher temperature physics related to laser cooling and trapping since it mostly classical. This lab will explore some experimental aspects of this field inspired by research in the Bakr and Cheuk experimental atomic physics groups at Princeton.

## Designing a Zeeman slower

In this section of the lab, you will design a Zeeman slower, a device that uses laser cooling to produce a slow, cold collimated atomic beam. The beam is cold in the sense that it has a narrow distribution of atomic velocities. It is slow in the sense that the average speed of the atoms exiting the slower is low, typically on the order of 10 m/s. For comparison, the particles in the air around you are moving at about 300 to 400 m/s.

The figure below shows a section of an ultrahigh vacuum chamber in the Bakr lab. 

![alt text](https://ultracold.princeton.edu/wp-content/uploads/sites/750/2020/09/zeeman_slower.png)

The section in the red box is an “oven” containing metallic lithium-6. It is heated to 620 K to achieve the desired vapor pressure of lithium. Some of the vapor passes through an aperture that produces a collimated atomic beam moving to the right side of the picture. Next, the beam enters the Zeeman slower section in the blue box consisting of a 30 cm long pipe surrounded by a set of coils outside the vacuum chamber. Current flowing in the coils produces a spatially varying magnetic field along the axis of the pipe. A laser beam propagates along the pipe’s axis coaligned with the atomic beam but traveling in the opposite direction. The laser beam exerts a force on the atoms to slow them down. 


### Exercise 1

**[10 points]**

*The normalized probability distribution for the speeds of the particles in an idealized gas is known as the Maxwell-Boltzmann distribution and is given by*

$f(v)=(m/2πkT)^{3/2} 4πv^2 e^{-mv^2/2kT}$, 

*where $m$ is the mass of the atoms in the gas and $T$ is the temperature of the gas in Kelvins.*

*Plot this distribution for a gas of lithium-6 at $T=300K$, $T=450 K$ and $T=620 K$. (Remember that all plots should be `publication ready,' as discussed in Lab 1---i.e., they should be properly labeled.) What is the most probable speed for the atoms at each of these temperatures?*

In [ ]:
# use this space to complete excercise 1

Lithium is an alkali atom with a single “valence” electron (an electron in its outer shell). In quantum mechanics, the energy of this electron is quantized, meaning it can only take certain values corresponding to the different orbital states this electron can occupy. In the lowest energy configuration (the ground state), the valence electron occupies the “2s” orbital, but the laser can induce transitions of this electron a higher energy “2p” orbital. This happens when an atom absorbs a bundle of energy known as a photon from the laser beam. The photon carries an energy  $hc/λ$ and a momentum $h/λ$ directed along the direction of propagation of the laser beam. Here h is a fundamental constant of nature known as Planck’s constant, c is the speed of light and $λ$ is the wavelength of the laser light used to induce the transition, in this case 671 nm. The corresponding frequency of the light is given by $f=c/λ$.

![alt text](https://ultracold.princeton.edu/wp-content/uploads/sites/750/2020/09/energy_levels.png)

Laser cooling works by bombarding an atom with a stream of photons to slow it down. Each time an atom absorbs a photon, it makes a transition from 2s to 2p and the momentum of the photon is transferred to the atom. A few hundred nanoseconds later, the electron undergoes a spontaneous emission event where it decays back to the ground state and reemits the photon in a random direction. Over hundreds of absorption and emission events, the momentum due to the emitted photon averages to zero while the momenta of the absorbed photons add up and decelerate the atom.

One thing might worry you in this story. What stops an atom from absorbing too many photons and speeding up in the opposite direction? To answer that, let’s go into a bit more detail. Atoms are very picky about the frequency of the photons they absorb. A simple classical model of a lithium atom consists of a valence electron attached to the nucleus with a spring (corresponding to the Coulomb force between them). The spring is taken to have zero equilibrium length. The laser’s electric field shakes the electron and as you learned from mass-spring systems, the response is only strong near resonance. Similarly, in the quantum picture, the electron’s probability to make the transition between 2s and 2p is only large if the energy of photon matches the resonance condition, i.e. it is close to the energy separation between the levels.

### Exercise 2 
**[10 points]**

*You learned in driven mass-spring systems that the width of the resonance is set by the damping rate of the oscillator. In the classical electron-on-a-spring model of the atom, this is provided by “radiation damping”. The oscillating electron radiates energy at a rate given by $P=e^2 a^2/(6πε_0 c^3)$. Here e is the charge of the electron, $\epsilon_0$ is a constant known as the permittivity of free space and $c$ is the speed of light. This formula for the radiated power is known as the Larmor formula and you will derive it in PHY 304. Estimate the linewidth of the atomic resonance of lithium-6 within this classical picture, looking up the values of any constants you need. The actual linewidth is $\Gamma=2π \times 6$ MHz. In the quantum picture, this corresponds to the spontaneous emission rate from the 2p state. Hint: Equate the power dissipated by radiation with the power dissipated due to damping by a simple harmonic oscillator.*    

In [ ]:
#Use this space to complete excercise 2

The second concept we need to recall is the Doppler effect. You might remember that an observer (in this case the atom) moving opposite to the direction of the wave will see its frequency shifted up, while an observer moving in the direction of the wave will see its frequency shifted down. Since the speed of the atoms is small compared to the speed of light, the magnitude of the shift can be approximated by $\Delta f=v/\lambda$ and corresponds to hundreds of MHz for atoms moving at thermal speeds (verify that!), i.e. it is large compared to the linewidth. Thus, to cool atoms with typical speeds coming out of the oven, one has to detune the laser frequency below the transition frequency for stationary atoms. For atoms with a certain velocity, the light will then appear on resonance. As you see, the Doppler effect ensures that only atoms moving towards the beam will experience a force due to the light.

### Exercise 3
**[10 points]**

*Write down a formula for the force exerted by the light on the atoms averaged over many scattering cycles. Assume a scattering rate $\gamma_p$.*

In [ ]:
#Use this space to complete excercise 3

The dependence of the scattering rate on the detuning from the atomic resonance is given by 

$$\gamma_p=\frac{s_0Γ}{2}\frac{1}{1+s_0+(2δ/Γ)^2},$$ 

where $\delta$ is the angular frequency detuning from resonance and the atomic linewidth is $\Gamma=2\pi \times 6$ MHz (both measured in rad/s) and $s_0$ is a dimensionless saturation parameter that depends on the intensity of the light (if the atom is on resonance and the light is very intense, $s_0$ is large and the scattering rate saturates to $Γ/2$). 

The average force on an atom, and hence its deceleration, is maximal if the light is seen on resonance by the moving atom. We can arrange that for a particular velocity class of atoms entering the slower by choosing an appropriate laser detuning. However, as these atoms slow down, their Doppler shift changes and they go off resonance. The Zeeman slower solves this problem by introducing a magnetic field that keeps the light on resonance as the velocity of the atoms is reduced. The magnetic field tunes the energy splitting between the 2s and 2p levels due to an effect known as the Zeeman effect. You can think of the “valence” electron as a tiny bar magnet whose energy is lowered (increased) by aligning (anti-aligning) with the magnetic field. The shift in the energy splitting relative to the energy splitting at zero field is given by $\mu_B B$, where $\mu_B$ is physical constant called the Bohr magneton and B is the magnetic field. The corresponding shift in the angular frequency of the transition is $2\pi\mu_B B/h$.


### Exercise 4
**[10 points]**

*Define $\delta_0$ to be the “bare” detuning of the laser (i.e. the detuning of the laser from resonance for a stationary atom in the absence of a magnetic field). Write down a formula for the effective detuning $\delta$ for an atom moving opposite to the direction of the laser beam with instantaneous speed $v$ at a position $z$. All frequencies are angular frequencies. What is the maximum deceleration $a_{max}$  achievable with a Zeeman slower? Derive a formula for the ideal Zeeman shift spatial dependence (and hence the magnetic field profile $B(z)$) required to keep the Zeeman slower as short as possible.*

In [ ]:
#Use this space to complete excercise 4

Any experimental imperfections in the magnetic field profile of the ideal slower will force atoms out of the slowing process. Therefore, it is more common to limit the deceleration to a constant acceleration $a=\eta a_{max}$ where $\eta<1$. In addition, as you can see from the picture of the slower, the coils are discrete leading to further deviations from the perfect field profile. 

### Exercise 5
**[20 points]**

In this part, you will simulate the dynamics of atoms in the actual Zeeman slower used in the Bakr lab. Import the data for the magnetic field profile along the slower and plot it. The first column is the distance from the start of the slower in meters and the second column is the magnetic field at that position in Tesla (the SI unit for magnetic field). Integrate Newton’s equations numerically and plot $v(z)$, the speed of an atom as a function of its position in the slower for three different initial speeds. You might find scipy's <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html">odeint</a> helpful for performing the numerical integration.

Assume a saturation parameter $s_0=4$ and a bare detuning $\delta_0=-2\pi \times 100$ MHz. What is the maximum speed of atoms that can be slowed with this device? How does it compare to the most probable speed of the atoms in the oven at the operational temperature?

In [ ]:
#Use this space to complete excercise 5

Once the atoms are cold enough, they can be captured in one of various traps that can be created using electromagnetic fields. One example of such a trap is the optical dipole trap created by focusing a laser beam. When the beam is very tightly focused so that it only traps at most a few atoms, such a trap is commonly referred to as an optical tweezer. The figure below is an optical image of atoms in a partially filled array of optical tweezers created in Jeff Thompson’s group in the EE department (see Physical Review Letters 122, 143002 (2019)). Each dot you see corresponds to the light scattered by a single $^{171}Yb$ atom and the tweezers are separated by a few micrometers . The goal of the experiment is to use the atoms for quantum computing. The Cheuk group does similar work with arrays of CaF molecules. A quantitative understanding of atom trapping in tweezers requires some quantum mechanics, but we can make again use of the classical mass-on-a-spring picture of the atom to get a semi-quantitative understanding of the physics.

![alt text](https://ultracold.princeton.edu/wp-content/uploads/sites/750/2020/09/tweezer_array.png)

Unlike the laser light used for cooling, the light used for optical trapping is very far detuned from the atomic resonance (i.e. $\delta \gg \Gamma$) and much more intense. In this regime, photon scattering by the atoms is negligible and the light exerts conservative forces on the atoms. When a “Gaussian” laser beam is focused by a lens, its intensity profile is given by

$I(r,z)= \frac{2P}{\pi w(z)^2} e^{-2r^2/w(z)^2}$,

where $P$ is the laser power, $z$ is the axial coordinate along the propagation direction of the beam with the origin chosen at the focus, $r$ is the radial coordinate orthogonal to it and $w(z)$ characterizes the size of the beam at position $z$ and is given by 

$w(z)=w_0 \sqrt{1+(\frac{z}{z_R})^2}$.

Here $w_0$ is called the waist and measures the size of the beam at the focus while the “Rayleigh range” $z_R$ is given by $z_R=πw_0^2/λ$. 

How does the light trap the atoms? Consider again the lithium atom modelled as a valence electron attached to the nucleus by a spring. The oscillating electric field $E(r,t)=E_0 (r)e^{-iωt}$ of the light exerts an oscillating force $F(r,t)=-eE(r,t)$ on the electron. Here $\omega$ is the angular frequency of the light. That leads to an oscillating charge that has a “dipole moment” $p=-ex$ where $x$ is the position of the electron relative to the nucleus. The proportionality constant between this moment and the field is known as the complex polarizability $\alpha(\omega)$, i.e.  $p=\alpha(\omega)E$. This induced dipole moment in turn interacts with the electric field and the time-averaged shift in the energy of the electron due to the light is given by $U(r,z)=-\frac{1}{2ε_0 c} \Re(α)  I(r,z)$. 


### Exercise 6
**[10 points]**

*Derive a formula for the complex polarizability of an atom like lithium-6 with resonance angular frequency $\omega_0$ and spontaneous emission rate $\Gamma$. Take the mass of the electron to be $m_e$. Hint: start by writing down the equation for a damped, driven harmonic oscillator and the solving it. Convince yourself that the laser frequency has to be tuned below the atom resonance to trap the atoms at the point of highest intensity of a Gaussian laser beam. Taylor expand the trapping potential about that point and find the angular frequency of small oscillations of the atom $\omega_r$ and $\omega_z$ in the $r$ and $z$ directions respectively.*

In [ ]:
#Use this space to complete excercise 6

Now it’s time to go into a real lab! Watch the video of Ben Spar, a graduate student in the Bakr group measuring the trapping frequency of a lithium-6 atom in an optical tweezer by introducing a small temporal modulation to the laser power, leading to heating of the atom and its escape from the trap at certain driving frequencies. Since the modulation affects the tweezer depth and the atoms sit at the center of the tweezer, one would not expect any energy transfer to a classical particle. However, the atoms are quantum mechanically spread over a "wavefunction" and the symmetric modulation drives a breathing mode of the wavefunction. The breathing resonance occurs at twice the trapping frequency. The measurements he has taken are provided. 

### Exercise 7
**[30 points]**

(a) Describe the experimental procedure in your own words. 

(b) Obtain the probability of survival of the atom at each modulation frequency and make a plot of the data. Include errorbars for the data points and explain how you obtained them. You might find <a href="https://matplotlib.org/api/_as_gen/matplotlib.pyplot.errorbar.html">plt.errorbar()</a> useful.

(c) Fit Lorentzian curves to the data to extract the axial and radial frequencies of the tweezer. Report the results of the fit for the trapping frequencies and the fit errors in these quantities. Remember that the atom loss occurs at a driving frequency that is twice the trapping frequency. Also remember that the frequencies you extract from the experiment are measured in Hz, while the angular frequencies you derived in the previous part are in rad/s.  You might find scipy's <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html">curve_fit</a> useful. 

(d) You should see that the Lorentzian fit used for extracting the radial trapping frequency is not so good. In fact, the data seems to indicate the presence of two peaks. Why do you think that might be? Ignore this subtlety for the rest of your data analysis.

(e) Find a way to extract the waist of the tweezer only using the measured axial and radial trapping frequencies. Use error propagation to obtain a statistical error on the extracted value of the waist. Note that the wavelength of the light used to create the tweezer is 830 nm.

(f) Next extract a value for the waist using a different method using the radial trapping frequency and the measured power, but not the axial frequency. Again obtain a statistical error on the waist. This method has a large potential systematic error. Based on the video, what is that? Is the value of the waist extracted with this method larger or smaller than the value from part (e)? Does that make sense?



In [ ]:
#Use this space to complete excercise 7